## Web Scraping: El Pais

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

### Obtain list of news from the coverpage

URL definition: 

In [2]:
# url definition
url = "https://elpais.com/elpais/inenglish.html"

List of news:

In [3]:
# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html5lib')

# News identification
coverpage_news = soup1.find_all('h2', class_='articulo-titulo')
len(coverpage_news)

67

Now we have a list in which every element is a news article:

In [4]:
coverpage_news[4]

<h2 class="articulo-titulo" itemprop="headline">
<a href="https://elpais.com/elpais/2019/12/26/inenglish/1577353137_122414.html">Politicians from all sides of the spectrum respond to King Felipe’s Christmas speech</a>
</h2>

### Let's extract the text from the articles:

First, we'll define the number of articles we want: 

In [5]:
number_of_articles = 5

In [6]:
# Empty lists for content, links and titles
news_contents = []
list_links = []
list_titles = []

for n in np.arange(0, number_of_articles):
    
    # only news articles (there are also albums and other things)
    if "inenglish" not in coverpage_news[n].find('a')['href']:  
        continue
    
    # Getting the link of the article
    link = coverpage_news[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title
    title = coverpage_news[n].find('a').get_text()
    list_titles.append(title)
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='articulo-cuerpo')
    x = body[0].find_all('p')
    
    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)
        
    news_contents.append(final_article)

Let's put them into:

- a dataset which will the input of the models (df_features)
- a dataset with the title and the link (df_show_info)

In [7]:
# df_features
df_features = pd.DataFrame(
     {'Article Content': news_contents 
    })

# df_show_info
df_show_info = pd.DataFrame(
    {'Article Title': list_titles,
     'Article Link': list_links})

In [8]:
df_features

,Article Content
0,At least three suspected members of the Main I...
1,Drowning was the sole cause of death for the t...
2,Ilyas El Masdouri knows what it feels like to ...
3,"Former Catalan premier, Carles Puigdemont, has..."
4,Spanish politicians from all sides of the poli...


In [9]:
df_show_info

,Article Title,Article Link
0,Three suspected Russian spies traveled to Barc...,https://elpais.com/elpais/2019/12/27/inenglish...
1,British victims who drowned in Costa del Sol r...,https://elpais.com/elpais/2019/12/27/inenglish...
2,Meet the migrants helping the homeless in the ...,https://elpais.com/elpais/2019/12/26/inenglish...
3,Ousted Catalan premier calls for European arre...,https://elpais.com/elpais/2019/12/27/inenglish...
4,Politicians from all sides of the spectrum res...,https://elpais.com/elpais/2019/12/26/inenglish...


### Time Elapsed

We are interested in how much time does the script takes to get the news because this will impact directly on user experience. For this, we'll put it all into a single function and then call it:

In [10]:
def get_news_elpais():
    
    # url definition
    url = "https://elpais.com/elpais/inenglish.html"
    
    # Request
    r1 = requests.get(url)
    r1.status_code

    # We'll save in coverpage the cover page content
    coverpage = r1.content

    # Soup creation
    soup1 = BeautifulSoup(coverpage, 'html5lib')

    # News identification
    coverpage_news = soup1.find_all('h2', class_='articulo-titulo')
    len(coverpage_news)
    
    number_of_articles = 5

    # Empty lists for content, links and titles
    news_contents = []
    list_links = []
    list_titles = []
    
    for n in np.arange(0, number_of_articles):

        # only news articles (there are also albums and other things)
        if "inenglish" not in coverpage_news[n].find('a')['href']:  
            continue

        # Getting the link of the article
        link = coverpage_news[n].find('a')['href']
        list_links.append(link)

        # Getting the title
        title = coverpage_news[n].find('a').get_text()
        list_titles.append(title)

        # Reading the content (it is divided in paragraphs)
        article = requests.get(link)
        article_content = article.content
        soup_article = BeautifulSoup(article_content, 'html5lib')
        body = soup_article.find_all('div', class_='articulo-cuerpo')
        x = body[0].find_all('p')

        # Unifying the paragraphs
        list_paragraphs = []
        for p in np.arange(0, len(x)):
            paragraph = x[p].get_text()
            list_paragraphs.append(paragraph)
            final_article = " ".join(list_paragraphs)

        news_contents.append(final_article)
        
    # df_features
    df_features = pd.DataFrame(
         {'Content': news_contents 
        })

    # df_show_info
    df_show_info = pd.DataFrame(
        {'Article Title': list_titles,
         'Article Link': list_links,
         'Newspaper': 'El Pais English'})
    
    return (df_features, df_show_info)

In [11]:
start = time.time()
x, y = get_news_elpais()
end =time.time()
te = end-start
print("The time elapsed is %f seconds" %(te))

The time elapsed is 6.523742 seconds


Seems like a fine number.